In [ ]:
import torch 
import os 
import pandas as pd 
import re 
import pickle 
from tqdm import tqdm 
from sentence_transformers import util
from numpy import dot
from numpy.linalg import norm 
from sentence_transformers import SentenceTransformer, util
import time

# device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"]= "0" 

with open('sbert_sts_training.pickle', 'rb') as pkl:
    defect_sbert = pickle.load(pkl)


In [ ]:
import pandas as pd 
import re 
import numpy as np


mode = 'target classification system'
filename='datset file'
readfile = pd.read_csv ('./data/fs_data/'+filename, encoding='utf-8')
readfile

In [ ]:
label = set(readfile['label'].to_list())
len(list(label))

label

In [ ]:

nv_label_df= pd.read_excel('anchor words list.xlsx', sheet_name=mode) 
nv_dict= nv_label_df.to_dict()
criteria_name = list(nv_dict.keys())
print(criteria_name)

criteria_keywords=[]
for k in nv_dict.values():
    temp =[ word for word in list(k.values()) if type(word) is not float]
    criteria_keywords.append(temp)
print("criteria_keywords: ",criteria_keywords)

nv_anchor_words=[]
for k, v in zip(criteria_name, criteria_keywords): 
    v.append(k)
    nv_anchor_words.append(' '.join(v))
    
print(nv_anchor_words)
len(nv_anchor_words)

In [ ]:
#Compute embeddings of defect complaint data 

complaints = readfile['complaint']
complaint_emb = defect_sbert.encode(complaints, convert_to_tensor=True, show_progress_bar = True)
print(complaint_emb.size())

criteria_name_emb = defect_sbert.encode(nv_anchor_words, convert_to_tensor=True, show_progress_bar = True)   #input_className
print(criteria_name_emb.size())

In [ ]:
from tqdm import tqdm 
from sentence_transformers import util

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(complaint_emb, criteria_name_emb)
print(cosine_scores.size())

#Find the pairs with the highest cosine similarity scores
max_cos = []
for i in tqdm(range(len(cosine_scores))):
    temp =[]
    for j in range(len(criteria_name_emb)):
        temp.append(cosine_scores[i][j])
    max_cos.append(temp.index(max(temp)))

print(len(max_cos))

In [ ]:
#class encoding 
ct_idx_dict = {(i):criteria_name[i] for i in range(0, len(criteria_name))}
print(ct_idx_dict)

map_to_criteria=[]
for i in max_cos: 
    for k, v in ct_idx_dict.items():
        if i == k:
            map_to_criteria.append(''.join(v))

In [ ]:
resultDF = readfile[['complaint', 'label']]
resultDF['pred_label']= map_to_criteria
resultDF

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

y_pred = resultDF['pred_label'].to_list()
y_ans = resultDF['label'].to_list()

print(classification_report(y_ans, y_pred))
print(precision_recall_fscore_support(y_ans, y_pred, average='micro'))

In [ ]:
criteria_name_A =['Cabinet', 
                    'Gas and fire system', 
                    'Condensation',
                    'Structural defect',
                    'Heating system',
                    'Leakage',
                    'Paperwork',
                    'Woodframe work',
                    'Masonry',
                    'Flooring',
                    'Stonework',
                    'Finish of sink',
                    'Airconditioning system',
                    'Appliance',
                    'Sanitary and plumbing',
                    'Electrical system',
                    'Doors and windows',
                    'Tiling',
                    'Communication system']

criteria_name_B=['Opening', 'Condensation', 'Stability', 'Leakage', 'Step', 'Detachement', 'Poor surface', 'Uninstallation', 
                  'Corrosion', 'Misalignment', 'Contamination', 'Disconnection', 'Out of orders', 'On/Off defect', 'Poor joint', 
                  'Scratch', 'Caulking defect', 'Crack', 'Broken']



In [ ]:
import seaborn as sns # used for plot interactive graph.
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

%matplotlib inline
plt.rc('font', family='Malgun Gothic')

conf_mat = confusion_matrix(y_ans, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, cmap="Blues", fmt='d',
            xticklabels=criteria_name_A, 
            yticklabels=criteria_name_A)
plt.ylabel('Actual', size=12)
plt.xlabel('Predicted', size=12)
plt.title("Confusion Matrix ", size=15);

## Fine-tuned Word2Vec Similarity 


In [ ]:
import re
import gensim
import pandas as pd
import numpy as np
import scipy.sparse as ss
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from scipy import spatial
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
%matplotlib inline

kow2v_model= 'ko_w2v_defect-0710-v2.bin' 
#feature vec = 200
kovec = Word2Vec.load('./ko-w2v/'+kow2v_model)


def getSentenceMeanEmbeddings (sentenceCorpus):
    sent_embdding_list =[]
    for line in sentenceCorpus:
        sen2vec = None
        count=0
        for word in line:
#             print(word)
            if word in kovec.wv.index_to_key:
                count +=1
                if sen2vec is None:
                    sen2vec = kovec.wv.get_vector(word)
                else: 
                    sen2vec = sen2vec+ kovec.wv.get_vector(word)
        if sen2vec is not None:
            sen2vec = sen2vec/count
            sent_embdding_list.append(sen2vec)
            
    return sent_embdding_list



# Fine-tuned_ Word2Vec model 
complaints = readfile['complaint']

complaint_emb_w2v = getSentenceMeanEmbeddings (complaints)
criteria_name_emb_w2v = getSentenceMeanEmbeddings(nv_anchor_words) # input_className ( MI 키워드 집합일때)

print(len(complaint_emb_w2v), len(criteria_name_emb_w2v))



In [ ]:
from tqdm import tqdm 
from sentence_transformers import util

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(complaint_emb_w2v, criteria_name_emb_w2v)
print(cosine_scores.size())

In [ ]:
#Find the pairs with the highest cosine similarity scores
max_cos = []
for i in tqdm(range(len(cosine_scores))):
    temp =[]
    for j in range(len(criteria_name_emb_w2v)):
        temp.append(cosine_scores[i][j])
#     print(temp)
    max_cos.append(temp.index(max(temp)))
#     print(temp.index(max(temp)))
#     break
print(len(max_cos))

In [ ]:
ct_idx_dict = {(i):criteria_name[i] for i in range(0, len(criteria_name))}
print(ct_idx_dict)

map_to_criteria=[]
for i in max_cos: 
    for k, v in ct_idx_dict.items():
        if i == k:
            map_to_criteria.append(''.join(v))
            
            
resultDF_w2v = readfile[['complaint', 'label']]
resultDF_w2v['pred_label']= map_to_criteria
resultDF_w2v

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

y_pred = resultDF_w2v['pred_label'].to_list()
y_ans = resultDF_w2v['label'].to_list()

print(classification_report(y_ans, y_pred))
print(precision_recall_fscore_support(y_ans, y_pred, average='micro'))